In [1]:
import pandas as pd

from src.explainer import valid_explainers_dico
from src.io import load_results

pd.options.plotting.backend = "plotly"
from src.scoring import get_score_df, get_eligible_points_df, get_summary_df

In [2]:
result_df = load_results()
# todo make it multi index
score_df = get_score_df(result_df)
eligible_points_df = get_eligible_points_df(result_df)
summary_df = get_summary_df(result_df, score_df, eligible_points_df)
print(summary_df.round(2))

                   time  score  eligible_points  percentage
baseline_random    0.00   4.37               10        0.44
saabas             0.03   5.04               10        0.50
tree_shap          0.02   6.06               10        0.61
sage               0.85   3.86                5        0.77
kernel_shap       21.22   9.13               10        0.91
lime              10.58   8.33               10        0.83
maple            545.88   8.00               10        0.80


In [3]:
result_df

,x0_plus_x1_distribution_non_uniform_stat_dep,cough_and_fever,cough_and_fever_10_90,x0_plus_x1_distribution_uniform_stat_dep,x0_plus_x1_distribution_non_uniform_stat_indep
baseline_random,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
saabas,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
tree_shap,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
sage,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
kernel_shap,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
lime,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...
maple,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_cough_more_important_than_fever...,{'score': {'is_feature_importance_symmetric': ...,{'score': {'is_feature_importance_symmetric': ...


In [27]:
def _round(x,n=0):
    if x is None:
        return None
    return round(x,n)

test_columns = []
for test_name in result_df.columns:
    for sub_test in result_df[test_name][0]['score'].keys():
        new_col = f'{test_name}__{sub_test}'
        test_columns.append(new_col)
        summary_df[new_col] =  [_round(dico.get('score', {}).get(sub_test, None))
                                                   for idx,dico in result_df[test_name].items()]
    summary_df[test_name + '__time'] =  [_round(dico.get('time', None)) for idx,dico in result_df[test_name].items()]

In [28]:
compare = ['sage', 'kernel_shap']  # , 'tree_shap'
assert all([e_name in valid_explainers_dico.keys() for e_name in compare])

In [58]:
df = summary_df.loc[compare].transpose().rename(index={'eligible_points':'_eligible_points',
                                                       'percentage':'_percentage',
                                                       'score':'_score',
                                                       'time':'_time',})
df.round(2)

,sage,kernel_shap
_time,0.85,21.22
_score,3.86,9.13
_eligible_points,5.00,10.00
_percentage,0.77,0.91
x0_plus_x1_distribution_non_uniform_stat_dep__is_feature_importance_symmetric,1.00,1.00
x0_plus_x1_distribution_non_uniform_stat_dep__is_attribution_values_symmetric,NaN,1.00
x0_plus_x1_distribution_non_uniform_stat_dep__time,0.00,1.00
cough_and_fever__is_feature_importance_symmetric,0.00,1.00
cough_and_fever__is_attribution_values_symmetric,NaN,1.00
cough_and_fever__time,0.00,6.00


In [60]:
_common_rows = set(df.loc[df.notna().apply(all,axis=1)].index)
common_tests = set(test_columns).intersection(_common_rows)

score_common_tests = df.loc[common_tests].sum(axis=0).rename('__score_common_tests')
df.loc['__score_common_tests'] = list(score_common_tests)

In [62]:
list_common_rows = list(_common_rows) + ['__score_common_tests']

In [63]:
df_common = df.loc[list_common_rows].sort_index()
df_common.round(2)

,sage,kernel_shap
__score_common_tests,4.00,5.00
_eligible_points,5.00,10.00
_percentage,0.77,0.91
_score,3.86,9.13
_time,0.85,21.22
cough_and_fever_10_90__is_cough_more_important_than_fever,1.00,1.00
cough_and_fever_10_90__time,0.00,7.00
cough_and_fever__is_feature_importance_symmetric,0.00,1.00
cough_and_fever__time,0.00,6.00
x0_plus_x1_distribution_non_uniform_stat_dep__is_feature_importance_symmetric,1.00,1.00


In [64]:
df_common.loc[common_tests]

,sage,kernel_shap
x0_plus_x1_distribution_uniform_stat_dep__is_feature_importance_symmetric,1.0,1.0
cough_and_fever_10_90__is_cough_more_important_than_fever,1.0,1.0
cough_and_fever__is_feature_importance_symmetric,0.0,1.0
x0_plus_x1_distribution_non_uniform_stat_indep__is_feature_importance_symmetric,1.0,1.0
x0_plus_x1_distribution_non_uniform_stat_dep__is_feature_importance_symmetric,1.0,1.0


In [ ]:
# todo https://www.quora.com/How-do-I-highlight-a-row-in-Pandas-DataFrame
# todo score sur les tests en commun